# Gerekli kütüphaneler

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format = '%(asctime)s %(message)s',
                    datefmt = '%m/%d/%Y %I:%M:%S %p',
                    filename = 'logs/1_web_scraping.log',
                    level=logging.DEBUG)

import warnings
warnings.filterwarnings('ignore')

# 1. Genel tabloyu çekelim.

In [2]:
def get_html(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [3]:
url_turk_dizileri_listesi = 'https://tr.wikipedia.org/wiki/T%C3%BCrk_dizileri_listesi'
content = get_html(url_turk_dizileri_listesi)
logging.debug('Dizi isimlerinin bulunduğu tablo wikipedia üzerinden çekildi.')
tables = content.find_all('table')
for table in tables:
    print(table.prettify())

<table class="box-Güncelle plainlinks metadata ambox ambox-content" role="presentation">
 <tbody>
  <tr>
   <td class="mbox-image">
    <div style="width:52px">
     <span typeof="mw:File">
      <a class="mw-file-description" href="/wiki/Dosya:Ambox_outdated_content2.svg">
       <img class="mw-file-element" data-file-height="775" data-file-width="775" decoding="async" height="42" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Ambox_outdated_content2.svg/42px-Ambox_outdated_content2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Ambox_outdated_content2.svg/63px-Ambox_outdated_content2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Ambox_outdated_content2.svg/84px-Ambox_outdated_content2.svg.png 2x" width="42"/>
      </a>
     </span>
    </div>
   </td>
   <td class="mbox-text">
    <div class="mbox-text-span">
     Bu maddenin daha doğru ve güvenilir bilgi sunması için
     <b>
      güncellenmesi
     </b>
     gerekmektedir.
     

# 2. Özellikleri seçerek tablo haline getirelim.

In [4]:
wiki_root_link = "https://tr.wikipedia.org/"

In [5]:
dizi_adlar = []
dizi_bolumler = []
dizi_sezonlar = []
dizi_durumlar = []
dizi_platformlar = []
dizi_linkler = []

In [6]:
tables = content.find_all('table', {'class': 'wikitable sortable'})
rows = table.find_all('tr')

for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            dizi_ad = cells[0].text.strip('\n')
            dizi_wiki_link = wiki_root_link + cells[0].find('a').get('href') if cells[0].find('a') else "?"
            dizi_bolum_sayisi = cells[1].text.strip('\n') if cells[1].text else "?"
            dizi_sezon_sayisi = cells[2].text.strip('\n') if cells[2].text else "?"
            dizi_durumu = cells[3].text.strip('\n') if cells[3].text else "?"
            dizi_platform = cells[4].text.strip('\n') if cells[4].text else "?"
            dizi_adlar.append(dizi_ad)
            dizi_bolumler.append(dizi_bolum_sayisi)
            dizi_sezonlar.append(dizi_sezon_sayisi)
            dizi_durumlar.append(dizi_durumu)
            dizi_platformlar.append(dizi_platform)
            dizi_linkler.append(dizi_wiki_link)

logging.debug('Tablodaki dizi linki, dizi adı, bölüm sayısı, sezon sayısı, durumu ve platformu bilgileri bir diziye aktarıldı.')

In [7]:
df = pd.DataFrame({'Dizi Linki': dizi_linkler,
                   'Dizi Adı': dizi_adlar,
                   'Bolum Sayısı': dizi_bolumler,
                   'Sezon Sayısı': dizi_sezonlar,
                   'Durumu': dizi_durumlar,
                   'Platform': dizi_platformlar})
logging.debug('Özellikleri veri seti haline getirildi.')
df.head()

,Dizi Linki,Dizi Adı,Bolum Sayısı,Sezon Sayısı,Durumu,Platform
0,https://tr.wikipedia.org//w/index.php?title=Ad...,Adsız Aşıklar,-,1,Yakında,Netflix
1,https://tr.wikipedia.org//w/index.php?title=Ai...,Aile Albümü,-,1,Yakında,atv
2,https://tr.wikipedia.org//w/index.php?title=Al...,Alı-Kara,-,1,Yakında,Belli değil
3,https://tr.wikipedia.org//w/index.php?title=Al...,Altay,-,1,Yakında,tabii
4,https://tr.wikipedia.org//w/index.php?title=Ar...,Arjen,-,1,Yakında,GAİN


# 3. Dizi sayfalarına giderek başka özellikleri de çekelim.

In [8]:
features = ['Tür', 'Uyarlama', 'Senarist', 'Yönetmen', 'Başrol', 'Besteci', 'Sezon sayısı', 'Bölüm sayısı', 
            'Yapımcı', 'Mekân', 'Görüntü yönetmeni', 'Gösterim süresi', 'Yapım şirketi', 'Kanal', 'Platform', 
            'Ses formatı', 'Yayın tarihi', 'Durumu']

In [9]:
def find_dizi_info(dizi_link):
    info = {
        'Tür': "?",
        'Uyarlama': "?",
        'Senarist': "?",
        'Yönetmen': "?",
        'Başrol': "?",
        'Besteci': "?",
        'Sezon sayısı': "?",
        'Bölüm sayısı': "?",
        'Yapımcı': "?",
        'Mekân': "?",
        'Görüntü yönetmeni': "?",
        'Gösterim süresi': "?",
        'Yapım şirketi': "?",
        'Kanal': "?",
        'Platform': "?",
        'Ses formatı': "?",
        'Yayın tarihi': "?",
        'Durumu': "?",      
    }
    
    if dizi_link != "?":
        dizi_wiki_page = get_html(dizi_link)
        table = dizi_wiki_page.find('table', {'class': 'infobox'})
        youtube_link = 1 if dizi_wiki_page.find('a', {'title': 'YouTube'}) else 0
        info['Youtube'] = youtube_link
        facebook_link = 1 if dizi_wiki_page.find('a', {'title': 'Facebook'}) else 0
        info['Facebook'] = facebook_link
        twitter_link = 1 if dizi_wiki_page.find('a', {'title': 'Twitter'}) else 0
        info['Twitter'] = twitter_link
        instagram_link = 1 if dizi_wiki_page.find('a', {'title': 'Instagram'}) else 0
        info['Instagram'] = instagram_link
        
    else:
        info['Youtube'] = 0
        info['Facebook'] = 0
        info['Twitter'] = 0
        info['Instagram'] = 0
        return info
    
    try:
        for index, tr in enumerate(table.find_all('tr')):
            try:
                tr_name = tr.find('th').get_text()
                #if index == 0:
                #    info['Dizi Adı'] = [tr_name]
                if tr_name in features:
                    info[tr_name] = ','.join([text for text in tr.find('td').stripped_strings])
            except:
                continue
    
        
    except:
        pass
        #print('Wikipedia sayfası açılmamış!')

    logging.info(f'{dizi_link} adresinden bilgiler çekildi.')
    return info

In [10]:
#find_dizi_info(dizi_linkler[220])

In [11]:
test_df = pd.DataFrame(columns=['Tür', 'Uyarlama', 'Senarist', 'Yönetmen', 'Başrol', 'Besteci', 'Sezon sayısı', 'Bölüm sayısı', 'Yapımcı', 
                                'Mekân', 'Görüntü yönetmeni', 'Gösterim süresi', 'Yapım şirketi', 'Kanal', 'Platform', 'Ses formatı', 
                                'Yayın tarihi', 'Durumu', 'Youtube', 'Facebook', 'Twitter', 'Instagram'])
test_df.head()

,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,...,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Durumu,Youtube,Facebook,Twitter,Instagram


In [12]:
for i in tqdm(range(len(dizi_linkler))):
    dizi_link = dizi_linkler[i]
    dizi_info = find_dizi_info(dizi_link)
    test_df.loc[len(test_df)] = [dizi_info[key] for key in dizi_info.keys()]

100%|███████████████████████████████████████████████████████████████████| 1523/1523 [17:33<00:00,  1.45it/s]


In [13]:
test_df.sample(5)

,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,...,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Durumu,Youtube,Facebook,Twitter,Instagram
38,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,0,0,0,0
45,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,0,0,0,0
166,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,0,0,0,0
1421,Dram,?,Sulhi Dölek,"Uğur Yücel,Orhan Oğuz,Türkan Derya","Türkan Şoray,Şener Şen,Tarık Papuççuoğlu,Güven...",?,3,37,Mustafa Oğuz,"Samatya,,,İstanbul",...,"Erler Film,Most Production","atv,(1998-2001)",?,Stereo,29 Ekim 1998 - 11 Ocak 2001,Sona erdi,0,0,0,0
500,"Aksiyon,Dram",?,"Gökhan Horzum,(1-4. sezon),Damla Serim,(3-4. s...","Sinan Öztürk,(1-4. sezon),Özgür Sevimli,(2. se...","Aras Bulut İynemli,Dilan Çiçek Deniz,Ercan Kes...",Toygar Işıklı,4,"131 (,bölümleri listesi,)","Kerem Çatay,(1-4. sezon),Pelin Diştaş (1-2. se...","İstanbul,Nevşehir,Paris,Berlin",...,Ay Yapım,Show TV,?,Stereo,23 Ekim 2017 - 7 Haziran 2021,Sona erdi,1,0,1,1


In [14]:
logging.debug('Çekilen veriler veri seti olarak kaydedildi.')
test_df.to_csv('data/not_processed.csv')